In [2]:
# prepare data for testing

import pandas as pd
import numpy as np
import os

# save excel file with labeled PDFs as dataframeconda
labeled_df = pd.read_excel('firstPDFsLabeled.xlsx')

#dataframe from labeled excel file of 449 PDFs
labeled_df.head()

,File name,Table type,table Format,How many people in WNT table,header outside of the table or not,WHO,Comments
0,DigiMV2020_2EE65ATJV7_0_65613902_Accountantsve...,4.0,NaN,NaN,NaN,Abdulhai,NaN
1,DigiMV2020_2EE65ATJV7_0_65613902_Jaarrekening_...,4.0,NaN,NaN,NaN,Abdulhai,NaN
2,DigiMV2020_2FJ775FMWQ_0_74967894_Jaarrekening_...,3.0,0,1.0,0.0,Abdulhai,NaN
3,DigiMV2020_2SFYKJN5QQ_0_73866121_Accountantsve...,4.0,NaN,NaN,NaN,Abdulhai,NaN
4,DigiMV2020_2SFYKJN5QQ_0_73866121_Jaarrekening_...,4.0,NaN,NaN,NaN,Abdulhai,NaN


In [3]:
# filter out PDFs where Table type is equal to 1
labeled_df_1 = labeled_df[labeled_df['Table type'] == 1]

labeled_df_1.shape

(104, 7)

In [4]:
# pdf to list of strings(1 string per page)
# doesn't work with scanned PDFs
# https://stackoverflow.com/questions/26494211/extracting-text-from-a-pdf-file-using-pdfminer-in-python
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from io import StringIO

def extract_pages_as_strings(pdf_filename):
    resource_manager = PDFResourceManager()
    pages_content = []

    for page in PDFPage.get_pages(open(pdf_filename, 'rb')):
        fake_file_handle = StringIO()
        converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
        page_interpreter = PDFPageInterpreter(resource_manager, converter)
        page_interpreter.process_page(page)
        text = fake_file_handle.getvalue()
        pages_content.append(text)
        converter.close()
        fake_file_handle.close()

    return pages_content

# Example usage
pdf_path = "Type3.pdf"
pages = extract_pages_as_strings(pdf_path)
print(len(pages))
# To access content of a specific page, for example, page 2:
page_2_content = pages[21]
print(page_2_content)


25
___________________________________________________________________________________________________________

___________________________________________________________________________________________________________

Amado  Dronten  B.V.,  Dronten
___________________________________________________________________________________________________________

17.  WNT-VERANTWOORDING  2020  AMADO  DRONTEN  B.V.

De  W NT  is  van  toepassing  op  Amado  Dronten  B.V.  Het  voor  Amado  Dronten  B.V.  toepasselijke
bezoldigingsmaximum  is  in  2020  €  111.000.

1.  Bezoldiging  topfunctionarissen

1a.  Leidinggevende  topfunctionarissen  met  dienstbetrekking  en  leidinggevende  topfunctionarissen  zonder
dienstbetrekking  vanaf  de  13e  maand  van  de  functievervulling

Leidinggevende  topfunctionarissen  met  dienstbetrekking  en  leidinggevende  topfunctionarissen  zonder
dienstbetrekking  vanaf  de  13e  maand  van  de  functievervulling  inclusief  degenen  die  op  grond  van  h

In [5]:
# simple approach with 'in' operator
# returns list of page numbers where both keywords are present

keywords = ["bezoldiging", "wnt"]

candidates = []
for i,text in enumerate(pages):
    contains_all_keywords = all(keyword.lower() in text.lower() for keyword in keywords)
    if contains_all_keywords:
        candidates.append(i)
        
        
print(candidates)

[10, 21]


lets test pyahocorasick lib for keywords search.

In [12]:
#install pyahocorasick
#%conda install -c conda-forge pyahocorasick -y

# https://stackoverflow.com/questions/54146342/how-to-find-multiple-strings-in-a-pdf-file-using-python

import ahocorasick
import re
import string

def find_keywords_in_pdf(pdf_filename, keywords):
    pages_content = extract_pages_as_strings(pdf_filename)
    A = ahocorasick.Automaton()
    for idx, key in enumerate(keywords):
        A.add_word(key, (idx, key))
    A.make_automaton()

    for i, page in enumerate(pages_content):
        for end_index, (insert_order, original_value) in A.iter(page):
            start_index = end_index - len(original_value) + 1
            print("Page %d : Found '%s' at index %d" % (i, original_value, start_index))


# Example usage
pdf_path = "Type3.pdf"
keywords = ["WNT", "bezoldiging"]
find_keywords_in_pdf(pdf_path, keywords)


    



Page 6 : Found 'bezoldiging' at index 1221
Page 10 : Found 'WNT' at index 2202
Page 10 : Found 'bezoldiging' at index 2254
Page 21 : Found 'WNT' at index 363
Page 21 : Found 'bezoldiging' at index 515
Page 21 : Found 'bezoldiging' at index 1600
Page 21 : Found 'bezoldiging' at index 2094
Page 21 : Found 'bezoldiging' at index 2294
Page 22 : Found 'bezoldiging' at index 951
